In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
from PIL import Image
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import auc, roc_curve
from tqdm import tqdm

import time

In [3]:
# Keras & Tensorflow
import keras
import tensorflow as tf
from keras import layers
from keras.models import Model
from keras.layers import Lambda, concatenate
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D , Conv2D , MaxPooling2D
from keras.layers import  Dropout , BatchNormalization , Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from keras.callbacks import Callback , ReduceLROnPlateau , ModelCheckpoint
from sklearn.metrics import cohen_kappa_score, accuracy_score
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy

In [4]:
benign_train = np.load('/content/drive/MyDrive/CSE499/numpy/benign_train.npy')
malign_train = np.load('/content/drive/MyDrive/CSE499/numpy/malign_train.npy')
benign_test = np.load('/content/drive/MyDrive/CSE499/numpy/benign_test.npy')
malign_test = np.load('/content/drive/MyDrive/CSE499/numpy/malign_test.npy')
print('Done Loaded :)')

Done Loaded :)


In [5]:
#Shape of our dataset
print(f'Benign Train:',benign_train.shape)
print(f'Malignant Train',malign_train.shape)
print(f'Benign Test:',benign_test.shape)
print(f'Malignant Test',malign_test.shape)

Benign Train: (3500, 224, 224, 3)
Malignant Train (3496, 224, 224, 3)
Benign Test: (1500, 224, 224, 3)
Malignant Test (1500, 224, 224, 3)


In [6]:
benign_train_label = np.zeros(len(benign_train),dtype=float)
malign_train_label = np.ones(len(malign_train),dtype=float)
benign_test_label = np.zeros(len(benign_test),dtype=float)
malign_test_label = np.ones(len(malign_test),dtype=float) 

X_train = np.concatenate((benign_train, malign_train), axis = 0)
Y_train = np.concatenate((benign_train_label, malign_train_label), axis = 0)
X_test = np.concatenate((benign_test, malign_test), axis = 0)
Y_test = np.concatenate((benign_test_label, malign_test_label), axis = 0)

s = np.arange(X_train.shape[0])
np.random.shuffle(s)
X_train = X_train[s]
Y_train = Y_train[s]

s = np.arange(X_test.shape[0])
np.random.shuffle(s)
X_test = X_test[s]
Y_test = Y_test[s]

Y_train = to_categorical(Y_train, num_classes= 2)
Y_test = to_categorical(Y_test, num_classes= 2)

In [7]:
print(f'X train shape:',X_train.shape)
print(f'X test shape:',X_test.shape)
print(f'Y train shape:',Y_train.shape)
print(f'Y test shape:',Y_test.shape)

X train shape: (6996, 224, 224, 3)
X test shape: (3000, 224, 224, 3)
Y train shape: (6996, 2)
Y test shape: (3000, 2)


In [8]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=10)

In [9]:
print(f'x train shape:',x_train.shape)
print(f'x val shape:',x_val.shape)
print(f'y train shape:',y_train.shape)
print(f'y val shape:',y_val.shape)

x train shape: (5596, 224, 224, 3)
x val shape: (1400, 224, 224, 3)
y train shape: (5596, 2)
y val shape: (1400, 2)


In [10]:
from tensorflow.keras.applications import InceptionV3

def inceptionV3_model(backbone , lr = 1e-4):
  modelInceptionV3 = Sequential()
  modelInceptionV3.add(backbone)
  modelInceptionV3.add(layers.GlobalAveragePooling2D())
  modelInceptionV3.add(layers.Dropout(0.5))
  modelInceptionV3.add(layers.BatchNormalization())
  modelInceptionV3.add(layers.Dense(2 , activation='softmax'))
  
  modelInceptionV3.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(learning_rate=lr),
        metrics=['accuracy']
    )
    
  return modelInceptionV3


inceptionV3 = InceptionV3(
    weights = 'imagenet',
    include_top = False,
    input_shape =(224 , 224 , 3)
)
# call the model 
modelInceptionV3 = inceptionV3_model(inceptionV3 , lr = 1e-4)
modelInceptionV3.build((None, 224, 224, 3))
modelInceptionV3.summary()

87924736/87910968 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 batch_normalization_94 (Bat  (None, 2048)             8192      
 chNormalization)                                                
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                         

In [11]:
# Train the student model with MobileNet
epochs = 20
batch_size = 64
student_his_inceptionV3 = modelInceptionV3.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val))

Epoch 1/20
88/88 [==============================] - 148s 1s/step - loss: 0.2429 - accuracy: 0.9089 - val_loss: 0.1966 - val_accuracy: 0.9164
Epoch 2/20
88/88 [==============================] - 97s 1s/step - loss: 0.0850 - accuracy: 0.9657 - val_loss: 0.1591 - val_accuracy: 0.9514
Epoch 3/20
88/88 [==============================] - 97s 1s/step - loss: 0.0366 - accuracy: 0.9873 - val_loss: 0.1387 - val_accuracy: 0.9536
Epoch 4/20
88/88 [==============================] - 97s 1s/step - loss: 0.0173 - accuracy: 0.9937 - val_loss: 0.1502 - val_accuracy: 0.9593
Epoch 5/20
88/88 [==============================] - 97s 1s/step - loss: 0.0131 - accuracy: 0.9954 - val_loss: 0.1725 - val_accuracy: 0.9593
Epoch 6/20
88/88 [==============================] - 97s 1s/step - loss: 0.0069 - accuracy: 0.9979 - val_loss: 0.2006 - val_accuracy: 0.9621
Epoch 7/20
88/88 [==============================] - 98s 1s/step - loss: 0.0050 - accuracy: 0.9982 - val_loss: 0.2114 - val_accuracy: 0.9600
Epoch 8/20
88/88 [=

In [12]:
Y_val_pred = modelInceptionV3.predict(x_val)
print(f'The InceptionV3 model Accuracy on the Validation Set:',accuracy_score(np.argmax(y_val, axis=1), np.argmax(Y_val_pred, axis=1)))

The InceptionV3 model Accuracy on the Validation Set: 0.96


In [13]:
#Now let's check my Y_test values
print(f'My Y_test values are:\n' ,Y_test)

My Y_test values are:
 [[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [14]:
#Now let's check my predcited values from X_test dataset	
import timeit

start = timeit.default_timer()
#Your statements here
y_pred = modelInceptionV3.predict(X_test)
print(f'My predicted Y_test values are:\n' ,y_pred)

stop = timeit.default_timer()
print('\nTime: ',stop - start,'sec')

My predicted Y_test values are:
 [[9.1223672e-05 9.9990880e-01]
 [2.4678528e-05 9.9997532e-01]
 [1.0000000e+00 5.4044604e-09]
 ...
 [4.0138661e-12 1.0000000e+00]
 [9.9999750e-01 2.5226316e-06]
 [3.0270861e-09 1.0000000e+00]]

Time:  20.906086125 sec


In [15]:
#Now let's check the accuracy between the original & predicted (Y_test , y_pred)
print(f'My accuracy on Student model with InceptionV3 on the Test set is:',accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

My accuracy on Student model with InceptionV3 on the Test set is: 0.995


In [16]:
# Print the classification report
print(classification_report( np.argmax(Y_test, axis=1), np.argmax(y_pred, axis=1)))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1500
           1       1.00      0.99      0.99      1500

    accuracy                           0.99      3000
   macro avg       1.00      0.99      0.99      3000
weighted avg       1.00      0.99      0.99      3000

